# Set Up

In [ ]:
import sys
from google.colab import drive
import os

drive.mount('/content/drive')
drive_path = os.path.join('drive', 'MyDrive', 'Colab Notebooks', 'Dacon', 'SentenceOrder') # 다른 드라이브에서 사용할 경우, 이 부분만 적절히 수정

if drive_path not in sys.path:
  sys.path.append(drive_path)
from utility import *

SEED = 42
set_all_seed(SEED)

CONFIG_PATH = drive_path + '/config.yaml'
config = load_config(CONFIG_PATH)

In [ ]:
with open(drive_path + '/dataset/first_augmented_data.pkl', 'rb') as f:
  first_augmented_data = pickle.load(f) # 첫 번째 증강 데이터
with open(drive_path + '/dataset/second_augmented_data.pkl', 'rb') as f:
  second_augmented_data = pickle.load(f) # 두 번째 증강 데이터
train = pd.read_csv(drive_path + '/dataset/train.csv').drop(columns = 'ID') # 훈련 데이터

# Create DataFrame from the Augmented Data

In [ ]:
# 증강 데이터 세트에서 오류가 있는 데이터 파악
# 예를 들어, 문장이 네 개가 아니거나 한자 및 특수 문자를 포함한 데이터는 오류로 판단

first_error_idx = make_error_idx(first_augmented_data)
second_error_idx = make_error_idx(second_augmented_data)

In [ ]:
# 오류 데이터 제외한 데이터를 이용하여 데이터 프레임 생성

first_df =  make_df_from_complete_text_list(first_augmented_data, first_error_idx)
second_df = make_df_from_complete_text_list(second_augmented_data, second_error_idx)

In [ ]:
# 두 데이터 프레임을 합친 증강 데이터 프레임 생성

augmented_df = pd.concat([first_df, second_df]).sample(frac=1, random_state=SEED).reset_index(drop=True)
augmented_df.to_csv(drive_path + '/dataset/augmented_df.csv', index=False)
augmented_df.head(5)

# Create DataFrame from the Original Dataset

In [ ]:
# 오리지널 데이터 세트로부터 데이터 세트 생성
# 새 데이터 세트에는 오리지널 데이터가 정렬된 상태

sentences = [[] for _ in range(4)]
sentence_order = []

for i, row in tqdm(train.iterrows()):
  for j in range(4):
    sentences[j].append(row.iloc[row[f'answer_{j}']])

In [ ]:
# 정렬된 오리지널 데이터 프레임 생성

ordered_train = make_df(sentences)
ordered_train.to_csv(drive_path + '/dataset/ordered_train.csv', index=False)
ordered_train.head(5)

# Create the Joint Dataset

In [ ]:
# 증강 데이터와 오리지널 데이터를 합친 조인트 데이터 프레임 생성

joint_df = pd.concat([ordered_train, augmented_df]).sample(frac=1, random_state=SEED).reset_index(drop=True)
joint_df.to_csv(drive_path + '/dataset/joint_df.csv', index=False)
joint_df.head(5)

In [ ]:
# 각 데이터 프레임의 길이 출력

print( "Length of the Original Dataset:", len(train) )
print( "Length of the Augmented Dataset", len(augmented_df) )
print( "Length of the Final Dataset:", len(joint_df) )

# Create the Randomized Final Dataset


In [ ]:
def create_randomized_dataset(df, random_numbers, sentences, answers):
  # 정렬된 네 개의 문장을 섞어 데이터를 생성하는 함수

  for _, row in tqdm(df.iterrows()):
    random.shuffle(random_numbers)
    for i, n in enumerate(random_numbers):
      sentences[i].append(row.iloc[n]) # 이 배열에는 무작위로 섞인 네 개의 문장을 저장
      answers[n].append(i) # 이 배열에는 정답 저장

In [ ]:
random_numbers = [0, 1, 2, 3] # 무작위로 섞일 문장의 순서이자 정답이 될 배열
sentences = [[] for _ in range(4)] # 네 개의 무작위로 섞인 문장이 담길 배열
answers = [[] for _ in range(4)] # 정답이 담길 배열

for _ in range(6): # 전체 데이터에 대해, 무작위 데이터 생성 6번 반복
  create_randomized_dataset(joint_df, random_numbers, sentences, answers)

In [ ]:
final_train_df = make_df(sentences, answers).drop_duplicates().sample(frac=1, random_state=SEED).reset_index(drop=True) # 무작위로 섞인 데이터에서 중복 데이터를 제거하여 데이터 프레임 생성
final_train_df.to_csv(drive_path + '/dataset/final_train_df.csv', index=False) # 최종 데이터 프레임 저장
final_train_df.head(5)
print("Length of the Final Train Dataset: ", len(final_train_df))